# Self Organising Map Challenge

## The Kohonen Network

The Kohonen Self Organizing Map (kohonen_som) provides a data visualization technique which helps to understand high dimensional data by reducing the dimensions of data to a map. kohonen_som also represents clustering concept by grouping similar data together.

Unlike other learning technique in neural networks, training a kohonen_som requires no target vector. A kohonen_som learns to classify the training data without any external supervision.

![Network](http://www.pitt.edu/~is2470pb/Spring05/FinalProjects/Group1a/tutorial/kohonen1.gif)

### Structure
A network has a width and a height that descibes the grid of nodes.  For example, the grid may be 4x4, and so there would be 16 nodes.

Each node has a weight for each value in the input vector.  A weight is simply a float value that the node multiplies the input value by to determine how influential it is (see below)

Each node has a set of weights that match the size of the input vector.  For example, if the input vector has 10 elements, each node would have 10 weights.

### Training 
To train the network

1. Each node's weights are initialized.
2. We enumerate through the training data for kohonen_some number of iterations (repeating if necessary).  The current value we are training against will be referred to as the `current input vector`
3. Every node is examined to calculate which one's weights are most like the input vector. The winning node is commonly known as the Best Matching Unit (BMU).
4. The radius of the neighbourhood of the BMU is now calculated. This is a value that starts large, typically set to the 'radius' of the lattice,  but diminishes each time-step. Any nodes found within this radius are deemed to be inside the BMU's neighbourhood.
5. Each neighbouring node's (the nodes found in step 4) weights are adjusted to make them more like the input vector. The closer a node is to the BMU, the more its weights get altered.
6. Go to step 2 until we've completed N iterations.
    

### Calculating the Best Matching Unit (BMU)

To determine the best matching unit, one method is to iterate through all the nodes and calculate the Euclidean distance between each node's weight vector and the current input vector. The node with a weight vector closest to the input vector is tagged as the BMU.

The Euclidean distance $\mathsf{distance}_{i}$ (from the input vector $V$ to the $i$'th node's weights $W_i$)is given as (using Pythagoras):

$$ \mathsf{distance}_{i}=\sqrt{\sum_{k=0}^{k=n}(V_k - W_{i_k})^2}$$

where V is the current input vector and $W_i$ is the node's weight vector.  $n$ is the size of the input & weight vector.

*Note*: $V$ and $W$ are vectors.  $V$ is the input vector, and $W_i$ is the weight vector of the $i$'th node.  $V_k$ and $W_{i_k}$ represent the $k$'th value within those vectors.  

The BMU is the node with the minimal distance for the current input vector

### Calculating the Neighbourhood Radius

The next step is to calculate which of the other nodes are within the BMU's neighbourhood. All these nodes will have their weight vectors altered.

First we calculate what the radius of the neighbourhood should be and then use Pythagoras to determine if each node is within the radial distance or not.

A unique feature of the Kohonen learning algorithm is that the area of the neighbourhood shrinks over time. To do this we use the exponential decay function:

Given a desired number of training iterations $n$:
$$n_{\mathsf{max iterations}} = 100$$

Calculate the radius $\sigma_t$ at iteration number $t$:

$$\sigma_t = \sigma_0 \exp\left(- \frac{t}{\lambda} \right) \qquad t = 1,2,3,4... $$

Where $\sigma_0$ denotes the neighbourhood radius at iteration $t=0$, $t$ is the current iteration. We define $\sigma_0$ (the initial radius) and $\lambda$ (the time constant) as below:

$$\sigma_0 = \frac{\max(width,height)}{2} \qquad \lambda = \frac{n_{\mathsf{max iterations}}}{\log(\sigma_0)} $$

Where $width$ & $height$ are the width and height of the grid.

### Calculating the Learning Rate

We define the initial leanring rate $\alpha_0$ at iteration $t = 0$ as:
$$\alpha_0 = 0.1$$

So, we can calculate the learning rate at a given iteration t as:

$$\alpha_t = \alpha_0 \exp \left(- \frac{t}{\lambda} \right) $$

where $t$ is the iteration number, $\lambda$ is the time constant (calculated above)
        
### Calculating the Influence

As well as the learning rate, we need to calculate the influence $\theta_t$ of the learning/training at a given iteration $t$.  

So for each node, we need to caclulate the euclidean distance $d_i$ from the BMU to that node.  Similar to when we calculate the distance to find the BMU, we use Pythagoras.  The current ($i$'th) node's x position is given by $x(W_i)$, and the BMU's x position is, likewise, given by $x(Z)$.  Similarly, $y()$ returns the y position of a node.

$$ d_{i}=\sqrt{(x(W_i) - x(Z))^2 + (y(W_i) - y(Z))^2} $$

Then, the influence decays over time according to:

$$\theta_t = \exp \left( - \frac{d_{i}^2}{2\sigma_t^2} \right) $$

Where $\sigma_t$ is the neighbourhood radius at iteration $t$ as calculated above. 

Note: You will need to come up with an approach to x() and y().


### Updating the Weights

To update the weights of a given node, we use:

$$W_{i_{t+1}} = W_{i_t} + \alpha_t \theta_t (V_t - W_{i_t})$$
        
So $W_{i_{t+1}}$ is the new value of the weight for the $i$'th node, $V_t$ is the current value of the training data, $W_{i_t}$ is the current weight and $\alpha_t$ and $\theta_t$ are the learning rate and influence calculated above.

*Note*: the $W$ and $V$ are vectors 

### 1. Kohonen Network Implementation
- We pass input layer size, output layer size (WxH), learning rate and number of iterations to the constructor

In [1]:
from app.algorithm.kohonen import Kohonen, NonVectorisedKohonen, OutputLayer
import matplotlib.pyplot as plt
import copy

async def kohonen_fit(output_layer, random, input_size, width, height, iterations, vectorised=True):
  kohonen_som = {}
  if vectorised:
    print("Running Kohonen (Vectorised Kohonen)...")
    kohonen_som = Kohonen(output_layer, random=random, input_size=input_size, width=width, height=height, iterations=iterations)
  else:
    print("Running Kohonen (Non-vectorised Kohonen)...")
    kohonen_som = NonVectorisedKohonen(output_layer, random=random, input_size=input_size, width=width, height=height, iterations=iterations)

  input_layer = kohonen_som.get_input_layer()
  output_layer = kohonen_som.get_output_layer()

  plt.figure()
  plt.axis("off")
  plt.imshow(input_layer.vectors, aspect="1")
  plt.title(f"Input Layer - RGB (1x{input_layer.num_colours})")
  plt.show(block=False)

  plt.figure()
  plt.axis("off")
  plt.imshow(output_layer.nodes, aspect="1")
  plt.title(f"Output Layer ({output_layer.width}x{output_layer.height})")
  plt.show(block=False)

  await kohonen_som.fit()
  return kohonen_som

In [2]:
external_output_layer = copy.deepcopy(OutputLayer(100, 100))

In [3]:
som = await kohonen_fit(copy.deepcopy(external_output_layer), random=True, input_size=20, width=100, height=100, iterations=1000)

AttributeError: 'coroutine' object has no attribute 'get_output_layer'

In [ ]:
await kohonen_fit(copy.deepcopy(external_output_layer), random=True, input_size=20, width=100, height=100, iterations=100, vectorised=False)